In [1]:
import os
import numpy as np
from PIL import Image
import glob
import pandas as pd

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.autograd import Variable

/home/pramod/.local/lib/python3.5/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
class DatasetProcessing(Dataset):
    def __init__(self, file_list, transform=None):
        self.img_filename = file_list
        self.transform = transform        

    def __getitem__(self, index):
        img = Image.open(self.img_filename[index])
        filename = self.img_filename[index]
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)        
        return img, filename
    def __len__(self):
        return len(self.img_filename)

In [139]:
# for testimgs
#DATA_PATH = './cats_dogs/'
DATA_PATH = './imgs/'

use_gpu = True

file_list = glob.glob(DATA_PATH+'*.jpg')

print(len(file_list))
file_list[0]

import torchvision.models as models
model = models.resnet101(pretrained=True)

"""
# remove last fully-connected layer -- for alexnet
new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
model.classifier = new_classifier
"""

# remove last fully-connected layer - for resnet like models 
modules=list(model.children())[:-1]
model=nn.Sequential(*modules)

#print(model)

# for evaluation 
model.eval()

if use_gpu:
    print('yes')
    model = model.cuda()

batch_size = 32
workers = 4


22
yes


In [140]:
#file_list[0]

a = file_list[0]
print(a)
print(a[len(DATA_PATH):])

./imgs/img3.jpg
img3.jpg


In [141]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize])

In [142]:
dset_test = DatasetProcessing(file_list, transformations)

test_loader = DataLoader(dset_test,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=4
                        )

dset_test.__len__()

22

In [143]:
epochs = 1
feature_df = pd.DataFrame()
feature_dict = {}

for epoch in range(epochs):
    ### feature extraction phase        
    for iter, testdata in enumerate(test_loader, 0):
        print("*********iter:" + str(iter) + ' *********')
        test_inputs, test_file_name = testdata
        
        if use_gpu:
            test_inputs = Variable(test_inputs.cuda())
        else: test_inputs = Variable(test_inputs)
            
        test_outputs = model(test_inputs)
        test_outputs = test_outputs.cpu().data.numpy()        
        
        # this is not required for the alexnet 
        # for resnet to squeeze the dimension from the avg pool layers
        test_outputs = np.squeeze(test_outputs)
        # sometimes due to the selection of a particular batch size 
        # the output becomes a 1d vector instead of bsize X dim 2d vector
        if (len(test_outputs.shape) != 2):
            test_outputs = np.reshape(test_outputs,(1, -1))
        
        for i in range(test_outputs.shape[0]):

            # prepare the saving path
            temp = test_file_name[i]
            temp = temp[len(DATA_PATH):]
            index = temp.find(".jpg")
            temp = temp[:index]
            feature_dict[str(temp)] = test_outputs[i,:]
            feature_df = pd.DataFrame(feature_dict)
            
            
            #save_file_name = SAVE_PATH + temp
            #print(save_file_name)
            
            #np.savetxt(save_file_name, test_outputs[i,:])
feature_df = feature_df.transpose()

*********iter:0 *********


In [145]:
feature_df.iloc[15:]

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
img1,0.075019,0.377636,0.286328,0.150281,1.027439,1.492099,0.251387,1.048187,0.917360,0.147455,...,0.300168,0.390299,0.439247,0.007019,1.983532,0.404270,0.149851,0.169104,0.324171,0.479206
img2,0.119926,0.078213,0.924031,0.804706,1.053693,0.397415,0.592862,0.581773,2.219676,0.198823,...,0.471248,1.345471,0.174861,0.153973,0.050629,0.318552,1.741183,1.029128,0.126983,0.321812
img3,0.310494,0.491740,0.698567,0.068430,0.379037,0.305540,0.141801,0.080034,0.395707,0.038466,...,0.089840,0.276242,0.488761,0.940670,0.040950,0.350040,0.047583,0.270020,0.290705,0.123781
img4,1.068086,0.311717,0.184385,0.105212,0.296597,0.302899,0.226477,0.699843,1.045524,0.091345,...,0.410153,0.497746,0.372124,0.426840,0.248001,1.506437,0.026825,0.127176,0.218100,0.102669
messi,0.292575,0.038869,0.463703,1.148192,0.358206,1.220238,0.683077,1.257175,0.566717,0.122433,...,3.579520,0.078485,0.410738,0.271121,0.148141,0.055995,0.454379,0.661424,0.256392,0.143756
person,0.346181,0.107922,1.039151,0.014061,0.334132,0.999782,0.246748,0.128843,0.189489,0.693904,...,0.665440,0.823805,0.806350,1.636157,0.319102,1.631242,1.096510,0.247501,0.468823,1.144589
scream,1.687908,0.728034,1.137804,1.696552,0.108497,0.243827,0.631486,0.273321,0.621843,0.157070,...,0.949710,0.316270,0.447035,0.247235,0.149831,0.135350,0.363467,0.841205,0.746149,0.379276


In [146]:
csv_file = "./test_imgs_features.csv"
feature_df.to_csv(csv_file)

## Clustering and Plotting

In [154]:
import sys
sys.path.insert(0,'/home/pramod/work/unsupervised_learning/cluster/')
import clusters

In [159]:
cluster_feats = clusters.Cluster(plot=True,K=2)

kmeans,kmeans_labels = cluster_feats.kmeans_sklearn(feature_df.iloc[:15].values)
gmm, gmm_labels, gmm_prob = cluster_feats.gmm_sklearn(feature_df.iloc[:15].values)

In [160]:
gmm_labels

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0])

In [161]:
kmeans_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1], dtype=int32)

In [162]:
gmm_prob.round(3)

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [163]:
X_Test = feature_df.iloc[15:]
probs = gmm.predict_proba(X_Test)
print(probs.round(3))

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
